In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

from src.ml import pipeline
from src.core import file_manager

In [ ]:
df_annotated = pipeline.get_annotated_df('use')

In [ ]:
random_distances_df  = df_annotated.sample(n=1000, random_state=42)

random_distances_df.head()

In [ ]:
random_distances_df['txt'].to_csv(file_manager.filename_from_data_dir('output/sentences_to_classify_all_distances.csv'), index=False)

In [ ]:
sentences_higher_distances = pd.read_csv(file_manager.filename_from_data_dir('output/sentences_to_classify.csv'))

In [ ]:
sentences_higher_distances.head()

In [ ]:
classified_sentences = sentences_higher_distances['txt'].to_numpy()

result = [s for s in random_distances_df['txt'].to_numpy() if s in classified_sentences]

len(result)

In [ ]:
df_classified_manual = pd.read_csv(file_manager.filename_from_data_dir('output/sentences_classified.csv'))

df_classified_manual['label_index'] = df_classified_manual['label'].map(pipeline.dict_labels)

df_classified_manual['embeddings'] = df_classified_manual['txt'].map(dict_embeddings)

df_classified_manual.drop(['correct_label', 'used_intents', 'intents'], axis=1).head()

In [ ]:
random_distances_df['manual_label'] = df_classified_manual['label'].to_numpy()

random_distances_df[['txt', 'intent', 'manual_label', 'distance']].head()

In [ ]:
differences = random_distances_df[random_distances_df.apply(lambda x: x['intent'] == x['manual_label'], axis=1)]

print(differences.describe())

differences[['txt', 'intent', 'manual_label', 'distance']].head()

In [ ]:
differences = random_distances_df[random_distances_df.apply(lambda x: x['intent'] != x['manual_label'], axis=1)]

print(differences.describe())

differences[['txt', 'intent', 'manual_label', 'distance']].head(50)

In [ ]:
df_to_predict_manual = pipeline.generate_df_from_x_y(
    x_data=df_classified_manual['embeddings'].to_numpy(),
    y_data=df_classified_manual['label_index'].to_numpy()
)


df_to_predict_manual.head(2)

In [ ]:
import h2o

h2o.init()

In [ ]:
model_path = file_manager.filename_from_data_dir('output/h2o/models/StackedEnsemble_AllModels_1_AutoML_1_20220428_200646')

leader_model = h2o.load_model(model_path)

leader_model

In [ ]:
hf_preds_manual= leader_model.predict(h2o.H2OFrame(df_to_predict_manual))

predicts_manual = hf_preds_manual.as_data_frame().predict.to_numpy()

correct_label_manual = df_to_predict_manual.label.to_numpy()

print(len(predicts_manual), len(correct_label_manual))

In [ ]:
correct_predict_manual = np.equal(predicts_manual, correct_label_manual).sum()

correct_predict_manual / len(correct_label_manual)